In [1]:
import taxcalc as tc
from policyengine_us import Microsimulation
from policyengine_us.model_api import *
import numpy as np
import pandas as pd

In [2]:
class TaxCalcVariableAlias(Variable):
    label = "TaxCalc Variable Alias"
    definition_period = YEAR
    entity = TaxUnit
    value_type = float


class tc_RECID(TaxCalcVariableAlias):
    def formula(tax_unit, period, parameters):
        return tax_unit("tax_unit_id", period)


class tc_MARS(TaxCalcVariableAlias):
    def formula(tax_unit, period, parameters):
        filing_status = tax_unit("filing_status", period).decode_to_str()
        CODE_MAP = {
            "SINGLE": 1,
            "JOINT": 2,
            "SEPARATE": 3,
            "HEAD_OF_HOUSEHOLD": 4,
            "WIDOW": 5,
        }
        return pd.Series(filing_status).map(CODE_MAP)


class taxcalc_extension(Reform):
    def apply(self):
        self.add_variables(
            tc_RECID,
            tc_MARS,
        )


sim = Microsimulation(reform=taxcalc_extension, dataset="enhanced_cps_2023")
df = pd.DataFrame()

for variable in sim.tax_benefit_system.variables:
    if variable.startswith("tc_"):
        df[variable[3:]] = sim.calculate(variable)

df

,RECID,MARS
0,101.0,1.0
1,201.0,1.0
2,301.0,5.0
3,302.0,5.0
4,401.0,2.0
...,...,...
155307,108897504.0,2.0
155308,108897600.0,2.0
155309,108897600.0,2.0
155310,108897704.0,2.0


In [3]:
class A:
    x = 1


class B(A):
    y = 1


{key: getattr(B, key) for key in dir(B)}

{'__class__': type,
 '__delattr__': <slot wrapper '__delattr__' of 'object' objects>,
 '__dict__': mappingproxy({'__module__': '__main__', 'y': 1, '__doc__': None}),
 '__dir__': <method '__dir__' of 'object' objects>,
 '__doc__': None,
 '__eq__': <slot wrapper '__eq__' of 'object' objects>,
 '__format__': <method '__format__' of 'object' objects>,
 '__ge__': <slot wrapper '__ge__' of 'object' objects>,
 '__getattribute__': <slot wrapper '__getattribute__' of 'object' objects>,
 '__gt__': <slot wrapper '__gt__' of 'object' objects>,
 '__hash__': <slot wrapper '__hash__' of 'object' objects>,
 '__init__': <slot wrapper '__init__' of 'object' objects>,
 '__init_subclass__': <function B.__init_subclass__>,
 '__le__': <slot wrapper '__le__' of 'object' objects>,
 '__lt__': <slot wrapper '__lt__' of 'object' objects>,
 '__module__': '__main__',
 '__ne__': <slot wrapper '__ne__' of 'object' objects>,
 '__new__': <function object.__new__(*args, **kwargs)>,
 '__reduce__': <method '__reduce__' o

In [4]:
input_data = tc.Records(df)
policy = tc.Policy()
simulation = tc.Calculator(records=input_data, policy=policy)

simulation.calc_all()

In [5]:
simulation.array("RECID")

array([      101,       201,       301, ..., 108897600, 108897704,
       108897800], dtype=int32)